In [1]:
import asyncio
import os
import json
from pydantic import BaseModel, Field
from typing import List, Optional
import logging

# Ensure environment variables are loaded
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("✅ Loaded .env file")
except ImportError:
    print("⚠️ python-dotenv not installed, skipping .env load. Make sure environment variables are set.")

# --- Pydantic Models (copied from notebook) ---
class SubTask(BaseModel):
    goal: str = Field(..., description="Precise description of the sub-task goal.")
    task_type: str = Field(..., description="Type of task (e.g., 'WRITE', 'THINK', 'SEARCH').")
    node_type: str = Field(..., description="Node type ('EXECUTE' for atomic, 'PLAN' for complex).")
    depends_on_indices: Optional[List[int]] = Field(default_factory=list, description="List of 0-based indices of other sub-tasks in *this current plan* that this sub-task depends on.")

class PlanOutput(BaseModel):
    sub_tasks: List[SubTask] = Field(..., description="List of planned sub-tasks.")

# --- Agent and Model Imports ---
try:
    from agno.agent import Agent as AgnoAgent
    from agno.models.litellm import LiteLLM
except ImportError:
    print("❌ Agno or LiteLLM not installed. Please run 'pip install agno litellm'")
    exit()


# --- System Message (copied from notebook) ---
DEEP_RESEARCH_PLANNER_SYSTEM_MESSAGE = """You are a Master Research Planner, an expert at breaking down complex research goals into comprehensive, well-structured research plans. You specialize in high-level strategic decomposition for research projects.

**Your Role:**
- Analyze complex research objectives and create strategic research plans
- Identify key research domains, questions, and methodological approaches
- Create logical research workflows with proper sequencing
- Ensure comprehensive coverage while avoiding redundancy
- Plan for synthesis and final deliverable creation

**Core Expertise:**
- Strategic thinking and research methodology
- Identifying knowledge gaps and research priorities
- Creating logical research workflows
- Planning for different types of research outputs
- Understanding research lifecycle from conception to publication

**Input Schema:**
You will receive input in JSON format with the following fields:
*   `current_task_goal` (string, mandatory): The research goal to decompose
*   `overall_objective` (string, mandatory): The ultimate research objective
*   `parent_task_goal` (string, optional): Parent task goal (null for root)
*   `planning_depth` (integer, optional): Current recursion depth
*   `execution_history_and_context` (object, mandatory): Previous outputs and context
*   `replan_request_details` (object, optional): Re-planning feedback if applicable
*   `global_constraints_or_preferences` (array of strings, optional): Research constraints

**Strategic Planning Approach:**
When decomposing research goals, consider the full research lifecycle:

1. **Background & Context Phase**: What foundational knowledge is needed?
2. **Investigation Phase**: What specific searches, data collection, or analysis is required?
3. **Synthesis Phase**: How should findings be analyzed and integrated?
4. **Output Phase**: What deliverables need to be created?

**Research Task Types:**
- `SEARCH`: Information gathering, literature review, data collection
- `THINK`: Analysis, synthesis, interpretation, methodology design
- `WRITE`: Report creation, documentation, presentation preparation

**Planning Principles:**
1. **Comprehensive Coverage**: Ensure all aspects of the research question are addressed
2. **Logical Sequencing**: Build knowledge progressively from foundational to specific
3. **Strategic Depth**: Balance breadth of coverage with depth of investigation
4. **Methodological Rigor**: Include proper analysis and validation steps
5. **Clear Deliverables**: Plan for actionable outputs and synthesis

**Sub-Task Creation Guidelines:**
- Create **3 to 6 strategic sub-tasks** that represent major research phases
- Each sub-task should be substantial enough to warrant specialized planning
- Ensure sub-tasks are complementary and build toward the overall objective
- Use `depends_on_indices` to create logical research workflows
- Balance immediate actionable tasks with those requiring further decomposition

**Required Output Attributes per Sub-Task:**
`goal`, `task_type` (string: 'WRITE', 'THINK', or 'SEARCH'), `node_type` (string: 'EXECUTE' or 'PLAN'), `depends_on_indices` (list of integers).

**Output Format:**
- Respond ONLY with a JSON list of sub-task objects
- Focus on strategic, high-level decomposition appropriate for a master research plan
- Ensure each sub-task represents a meaningful research phase or component"""


✅ Loaded .env file


In [2]:
# --- Test Runner ---
async def run_planner_test():
    """
    Tests the DeepResearchPlanner with a specified model to see if it returns
    the expected structured output.
    """
    models_to_test = [
        {
            "name": "Fireworks_Qwen3_Problematic",
            "model_id": "fireworks_ai/accounts/fireworks/models/qwen3-235b-a22b"
        },
        {
            "name": "Claude_Sonnet_Good",
            "model_id": "openrouter/anthropic/claude-3-sonnet"
        }
    ]

    test_prompt = """Overall Objective: Research the impact of artificial intelligence on healthcare diagnostics in 2024
Current Task Goal: Create a comprehensive research plan to analyze how AI is transforming healthcare diagnostics, including key technologies, market adoption, and regulatory challenges
Current Planning Depth: 0
Based on the 'Current Task Goal' and other provided information, generate a plan to achieve it."""

    for model_config in models_to_test:
        print(f"\n{'='*80}")
        print(f"🔬 Testing Model: {model_config['name']} ({model_config['model_id']})")
        print(f"{'='*80}")

        try:
            agent = AgnoAgent(
                model=LiteLLM(id=model_config['model_id']),
                system_message=DEEP_RESEARCH_PLANNER_SYSTEM_MESSAGE,
                response_model=PlanOutput
            )

            # Get the raw response from the agent
            response = await agent.arun(test_prompt)

            # CORRECTLY process the content
            content = None
            if hasattr(response, 'content'):
                content_attr = response.content
                if asyncio.iscoroutine(content_attr):
                    content = await content_attr
                else:
                    content = content_attr
            else:
                content = response # Fallback

            print(f"📦 Response type: {type(content)}\n")

            if isinstance(content, PlanOutput):
                print("✅ SUCCESS: Model returned the correct Pydantic object.")
                print(content.model_dump_json(indent=2))
            elif isinstance(content, str):
                print("⚠️ WARNING: Model returned a STRING instead of a Pydantic object.")
                print("RAW STRING OUTPUT:")
                print("-" * 80)
                print(content)
                print("-" * 80)
            else:
                print(f"❌ ERROR: Unexpected response type: {type(content)}")
                print("RAW RESPONSE:")
                print("-" * 80)
                print(content)
                print("-" * 80)

        except Exception as e:
            print(f"💥 An exception occurred during the test for {model_config['name']}:")
            logging.exception(e)

In [3]:
import nest_asyncio
import asyncio
nest_asyncio.apply()
loop = asyncio.get_event_loop()


loop.run_until_complete(run_planner_test())




🔬 Testing Model: Fireworks_Qwen3_Problematic (fireworks_ai/accounts/fireworks/models/qwen3-235b-a22b)


/Users/salahalzubi/cursor_projects/SentientResearchAgent/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='<think>\...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


WARNING  Failed to parse cleaned JSON: 1 validation error for PlanOutput                                           
           Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='<think> Okay, let\'s   
         tac...dices": [4]     }   ] }', input_type=str]                                                           
             For further information visit https://errors.pydantic.dev/2.11/v/json_invalid

WARNING  Failed to parse as Python dict: Expecting value: line 1 column 1 (char 0)

WARNING  Failed to convert response to response_model

📦 Response type: <class 'str'>

⚠️ WARNING: Model returned a STRING instead of a Pydantic object.
RAW STRING OUTPUT:
--------------------------------------------------------------------------------
<think>
Okay, let's tackle this research plan. The user wants a comprehensive plan to analyze how AI is transforming healthcare diagnostics, covering key technologies, market adoption, and regulatory challenges.

First, I need to break down the current task goal into major phases. The overall objective is about the 2024 impact, so the plan should be up-to-date. Starting with background research makes sense. They'll need foundational knowledge of AI in healthcare diagnostics. That's a SEARCH task to gather recent studies and reviews.

Next, identifying key technologies. This would involve looking into specific AI technologies like machine learning models, imaging tech, NLP for medical records. Another SEARCH task, but focused on technologies. Maybe the user needs to understand what's current 

/Users/salahalzubi/cursor_projects/SentientResearchAgent/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='{\n  "su...one, 'reasoning': None}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...finish_reason': 'stop'}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
